In [7]:
import pymysql
import pandas as pd
import sqlalchemy
import logging
import re
from typing import Dict, List, Optional, Tuple
from sqlalchemy import create_engine, types
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

In [2]:
host= 
port =  
user = 
password = 
database = 'policy'


In [3]:
ST = datetime.now()
print(ST)

2024-11-13 02:11:29.553943


In [20]:
class PolicyDataProcessor:
    def __init__(self, db_config: Dict):
        """
        데이터베이스 설정 및 로깅 초기화
        """
        self.db_config = db_config
        self.engine = create_engine(
            f"mysql+pymysql://{db_config['user']}:{db_config['password']}@"
            f"{db_config['host']}:{db_config.get('port', 3306)}/{db_config['database']}"
        )
        
        logging.basicConfig(
            level=logging.DEBUG,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )

    def fetch_data(self) -> pd.DataFrame:
        """RDS에서 데이터 가져오기"""
        try:
            query = """
                SELECT `idx`, `target`, `condition`, `content`
                FROM policy_detail_cleaned
            """
            df = pd.read_sql(query, self.engine)
            logging.info(f"Successfully fetched {len(df)} records from database")
            return df
        except Exception as e:
            logging.error(f"Error fetching data: {e}")
            raise

    def extract_age(self, texts: List[str]) -> Tuple[int, int]:
        """
        여러 텍스트 필드에서 최소/최대 나이 정보 추출
        Returns: Tuple[min_age, max_age]
        """
        min_age = 0
        max_age = 200  # 실질적인 최대 나이 설정
        
        for text in texts:
            if pd.isna(text):
                continue
            
            # 나이 범위 패턴 (확장)
            # 1) 숫자와 숫자 사이에 다양한 구분자가 있는 경우
            range_patterns = [
                r'(?:만\s*)?(\d+)세?\s*(?:[~＊∼-]|[부]터)\s*(?:만\s*)?(\d+)세?',  # 19세~49세, 19세부터49세
                r'(?:만\s*)?(\d+)\s*[~＊∼-]\s*(?:만\s*)?(\d+)세?',                # 19~49세
                r'(?:만\s*)?(\d+)세?부터\s*(?:만\s*)?(\d+)세?(?:까지)?',          # 19세부터 49세까지
            ]
            
            found_range = False
            for pattern in range_patterns:
                range_matches = re.finditer(pattern, text)
                for match in range_matches:
                    curr_min = int(match.group(1))
                    curr_max = int(match.group(2))
                    min_age = max(min_age, curr_min)
                    max_age = min(max_age, curr_max)
                    found_range = True
                    break
                if found_range:
                    break
                    
            if found_range:
                continue
            
            # 최소 나이 패턴 (예: 19세 이상, 만 19세 이상)
            min_matches = re.finditer(r'(?:만\s*)?(\d+)세?\s*이상', text)
            for match in min_matches:
                curr_min = int(match.group(1))
                min_age = max(min_age, curr_min)
            
            # 최대 나이 패턴
            # 1) X세 이하/미만
            max_matches_after = re.finditer(r'(?:만\s*)?(\d+)세?\s*(?:이하|미만)', text)
            for match in max_matches_after:
                curr_max = int(match.group(1))
                if '미만' in match.group():
                    curr_max -= 1
                max_age = min(max_age, curr_max)
                
            # 2) 이하/미만 X세
            max_matches_before = re.finditer(r'(?:이하|미만)\s*(?:만\s*)?(\d+)세?', text)
            for match in max_matches_before:
                curr_max = int(match.group(1))
                if '미만' in match.group():
                    curr_max -= 1
                max_age = min(max_age, curr_max)
                    
            # 단일 나이 패턴 (예: 만 19세)
            single_matches = re.finditer(r'(?<![\d~\-])(?:만\s*)?(\d+)세(?!\s*(?:이상|이하|미만|까지|부터))', text)
            for match in single_matches:
                age = int(match.group(1))
                if min_age == 0:  # 최소 나이가 설정되지 않은 경우에만
                    min_age = age
                max_age = min(max_age, age)
                
        # 나이 정보가 없거나 논리적이지 않은 경우 기본값 반환
        if min_age == 0 and max_age == 200:
            return (0, 0)
        if min_age > max_age:
            return (0, 0)
            
        return (min_age, max_age)

    def extract_region(self, texts: List[str]) -> str:
        """여러 텍스트 필드에서 지역 정보 추출"""
        regions = {
            '서울': ['서울', '서울시', '서울특별시'],
            '부산': ['부산', '부산시', '부산광역시'],
            '대구': ['대구', '대구시', '대구광역시'],
            '인천': ['인천', '인천시', '인천광역시'],
            '광주': ['광주', '광주시', '광주광역시'],
            '대전': ['대전', '대전시', '대전광역시'],
            '울산': ['울산', '울산시', '울산광역시'],
            '세종': ['세종', '세종시', '세종특별시'],
            '경기': ['경기', '경기도'],
            '강원': ['강원', '강원도'],
            '충북': ['충북', '충청북도'],
            '충남': ['충남', '충청남도'],
            '전북': ['전북', '전라북도'],
            '전남': ['전남', '전라남도'],
            '경북': ['경북', '경상북도'],
            '경남': ['경남', '경상남도'],
            '제주': ['제주', '제주도', '제주특별자치도']
        }
        
        found_regions = set()
        for text in texts:
            if pd.isna(text):
                continue
                
            for region, keywords in regions.items():
                if any(keyword in text for keyword in keywords):
                    found_regions.add(region)
                    
        return ','.join(sorted(found_regions)) if found_regions else '전체'

    def extract_job(self, texts: List[str]) -> str:
        """여러 텍스트 필드에서 직업 정보 추출"""
        job_keywords = {
            '대학생': ['대학생', '학생', '재학생', '대학원생'],
            '직장인': ['직장인', '근로자', '재직자', '임금근로자', '급여생활자'],
            '창업자': ['창업자', '창업기업', '사업자', '예비창업자', '신규창업'],
            '구직자': ['구직자', '취업준비생', '취준생', '미취업자', '실업자'],
            '자영업자': ['자영업자', '소상공인', '개인사업자'],
            '농업인': ['농업인', '농민', '축산인', '어업인', '임업인'],
            '프리랜서': ['프리랜서', '특수고용직', '특수형태근로자']
        }
        
        found_jobs = set()
        for text in texts:
            if pd.isna(text):
                continue
                
            for job, keywords in job_keywords.items():
                if any(keyword in text for keyword in keywords):
                    found_jobs.add(job)
                    
        return ','.join(sorted(found_jobs)) if found_jobs else '전체'

    def process_fields(self, df: pd.DataFrame) -> pd.DataFrame:
        """여러 컬럼에서 나이, 지역, 직업 정보 추출"""
        try:
            processed_df = df.copy()
            
            # 나이 정보 추출 (최소/최대)
            age_info = processed_df.apply(
                lambda row: self.extract_age([row['target'], row['condition'], row['content']]), 
                axis=1
            )
            processed_df['min_age'], processed_df['max_age'] = zip(*age_info)
            
            # 지역 정보 추출
            processed_df['region'] = processed_df.apply(
                lambda row: self.extract_region([row['target'], row['condition'], row['content']]), 
                axis=1
            )
            
            # 직업 정보 추출
            processed_df['job'] = processed_df.apply(
                lambda row: self.extract_job([row['target'], row['condition'], row['content']]), 
                axis=1
            )
            
            logging.info("Successfully processed age, region, and job fields")
            return processed_df
            
        except Exception as e:
            logging.error(f"Error processing fields: {e}")
            raise

    def save_to_db(self, df: pd.DataFrame, table_name='policy_detail_filter') -> None:
        """처리된 데이터를 RDS에 저장"""
        try:
            df['idx'] = df['idx'].astype(int)
            df.to_sql(
                name=table_name,
                con=self.engine,
                index=False,
                if_exists='replace',
                chunksize=1000,
                dtype={
                    'idx': types.BigInteger,
                    'min_age': types.Integer,
                    'max_age': types.Integer,
                    'region': types.String(length=100),
                    'job': types.String(length=100)
                }
            )
            logging.info(f"Successfully saved {len(df)} records to {table_name}")
        except Exception as e:
            logging.error(f"Error saving data: {e}")
            raise

def main():
    db_config = {
        'host': host,
        'user': user,
        'password': password,
        'database': database
    }
    
    try:
        processor = PolicyDataProcessor(db_config)
        raw_df = processor.fetch_data()
        processed_df = processor.process_fields(raw_df)
        processor.save_to_db(processed_df)
        logging.info("Data processing completed successfully")
        
    except Exception as e:
        logging.error(f"Error in main process: {e}")
        raise

if __name__ == "__main__":
    main()

2024-11-13 02:19:14,896 - root - INFO - Successfully fetched 111 records from database
2024-11-13 02:19:14,968 - root - INFO - Successfully processed age, region, and job fields
2024-11-13 02:19:17,161 - root - INFO - Successfully saved 111 records to policy_detail_filter
2024-11-13 02:19:17,161 - root - INFO - Data processing completed successfully


In [21]:
ET = datetime.now()
print(ET-ST)

0:07:47.617580
